In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# System imports
import os
import sys
import yaml

# External imports
import matplotlib.pyplot as plt
import scipy as sp
from sklearn.metrics import auc
import numpy as np
import pandas as pd
import seaborn as sns
import torch
from pytorch_lightning.loggers import TensorBoardLogger, WandbLogger
from pytorch_lightning import Trainer
import wandb
import math
sys.path.append('../..')

from LightningModules.GNNClustering.Models.gnn_clustering import InteractionGNN

device = "cuda" if torch.cuda.is_available() else "cpu"
from pytorch_lightning.callbacks import ModelCheckpoint

In [3]:
def kaiming_init(model):
    for name, param in model.named_parameters():
        if name.endswith(".bias"):
            param.data.fill_(0)
        elif name.startswith("layers.0"):  # The first layer does not have ReLU applied on its input
            param.data.normal_(0, 1 / math.sqrt(param.shape[1]))
        else:
            param.data.normal_(0, math.sqrt(2) / math.sqrt(param.shape[1]))

In [4]:
def load_from_pretrained(model, path):
    
    checkpoint = torch.load(path)
    state_dict = checkpoint["state_dict"]
    model.load_state_dict(state_dict, strict=False)
    del state_dict
    
    return model

## Construct PyLightning model

In [5]:
with open("gnn_clustering_default.yaml") as f:
    hparams = yaml.load(f, Loader=yaml.FullLoader)

In [6]:
if hparams["use_toy"]:
    hparams["regime"] = []
    hparams["spatial_channels"] = 2
    hparams["clusters"] = int(1.0*hparams["num_tracks"])
model = InteractionGNN(hparams)
# model = load_from_pretrained(model, "/global/cfs/cdirs/m3443/usr/ryanliu/ITk_gnn_clustering/ITk_gnn_clustering/392pd7d9/checkpoints/last.ckpt")

## Metric Learning

In [7]:
from pytorch_lightning.callbacks import ModelCheckpoint

checkpoint_callback = ModelCheckpoint(
    monitor='eff',
    mode="max",
    save_top_k=2,
    save_last=True)

In [ ]:
# kaiming_init(model)

logger = WandbLogger(project="ITk_gnn_clustering")
trainer = Trainer(gpus=1, max_epochs=hparams["max_epochs"], logger=logger, num_sanity_val_steps=2, callbacks=[checkpoint_callback], log_every_n_steps = 50, default_root_dir="/global/cfs/cdirs/m3443/usr/ryanliu/ITk_gnn_clustering/")
trainer.fit(model)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: liuryan30 (exatrkx). Use `wandb login --relogin` to force relogin


GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name               | Type             | Params
--------------------------------------------------------
0 | cos                | CosineSimilarity | 0     
1 | node_encoder       | Sequential       | 100 K 
2 | edge_encoder       | Sequential       | 101 K 
3 | assignment_network | Sequential       | 126 K 
4 | ignn_blocks        | ModuleList       | 1.5 M 
5 | hgnn_blocks        | ModuleList       | 4.9 M 
--------------------------------------------------------
6.6 M     Trainable params
0         Non-trainable params
6.6 M     Total params
26.212    Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

/global/homes/r/ryanliu/.conda/envs/rapids/lib/python3.8/site-packages/torch_geometric/deprecation.py:12: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)
/global/homes/r/ryanliu/.conda/envs/rapids/lib/python3.8/site-packages/pytorch_lightning/utilities/data.py:72: UserWarning: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 678. To avoid any miscalculations, use `self.log(..., batch_size=batch_size)`.
  warning_cache.warn(
/global/homes/r/ryanliu/.conda/envs/rapids/lib/python3.8/site-packages/pytorch_lightning/utilities/data.py:72: UserWarning: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 747. To avoid any miscalculations, use `self.log(..., batch_size=batch_size)`.
  warning_cache.warn(


Training: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

## Sweep

In [5]:
run_name = input()

 eff&pur


In [6]:
with open("gnn_clustering_sweep.yaml") as f:
        sweep_hparams = yaml.load(f, Loader=yaml.FullLoader)
with open("gnn_clustering_default.yaml") as f:
        default_hparams = yaml.load(f, Loader=yaml.FullLoader)

In [7]:
sweep_configuration = {
    "name": run_name,
    "project": "ITk_barrell_gnn_embedding",
    "metric": {"name": "pur", "goal": "maximize"},
    "method": "grid",
    "parameters": sweep_hparams
}

In [8]:
def training():
    wandb.init()
    hparams = {**default_hparams, **wandb.config}
    if hparams["use_toy"]:
        hparams["regime"] = []
        hparams["spatial_channels"] = 2
        hparams["clusters"] = hparams["num_tracks"]
    model = InteractionGNN(hparams)
    model = load_from_pretrained(model, "/global/cfs/cdirs/m3443/usr/ryanliu/ITk_gnn_clustering/ITk_gnn_clustering/392pd7d9/checkpoints/last.ckpt")
    # kaiming_init(model)
    
    checkpoint_callback = ModelCheckpoint(
        monitor='eff',
        mode="max",
        save_top_k=2,
        save_last=True)

    logger = WandbLogger()
    trainer = Trainer(gpus=1, max_epochs=default_hparams["max_epochs"], log_every_n_steps = 50, logger=logger, callbacks=[checkpoint_callback], default_root_dir="/global/cfs/cdirs/m3443/usr/ryanliu/ITk_gnn_clustering/")
    trainer.fit(model)

In [ ]:
sweep_id = wandb.sweep(sweep_configuration, project = "ITk_barrel_gnn_clustering")

# run the sweep
wandb.agent(sweep_id, function=training)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Create sweep with ID: 0xvved9b
Sweep URL: https://wandb.ai/exatrkx/ITk_barrel_gnn_clustering/sweeps/0xvved9b


wandb: Agent Starting Run: xt3ul3nj with config:
wandb: 	toy_eff: 0.8
wandb: 	toy_pur: 1
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: exatrkx (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.12.15 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/global/homes/r/ryanliu/.conda/envs/pyg2/lib/python3.8/site-packages/pytorch_lightning/loggers/wandb.py:341: UserWarning: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
  rank_zero_warn(
wandb: WARNING Config item 'toy_eff' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'toy_pur' was locked by 'sweep' (ignored update).

  | Name               | Type             | Params
--------------------------------------------------------
0 | cos                | CosineSimilarity | 0     
1 | node_encoder       | Sequential       | 100 K 
2 | edge_encoder       | Sequential       | 101 K 
3 | assignment_network | Sequential       | 126 K 
4 | ignn_blocks        | ModuleList       | 1.5 M 
5 

Validation sanity check:   0%|          | 0/2 [00:00<?, ?it/s]

...

Epoch 25:  98%|█████████▊| 1000/1020 [02:56<00:03,  5.68it/s, loss=0.985, v_num=l3nj]
Validating: 0it [00:00, ?it/s]
Validating:   0%|          | 0/20 [00:00<?, ?it/s]

/global/u2/r/ryanliu/Tracking-ML-Exa.TrkX/Pipelines/Common_Tracking_Example/notebooks/ITk/../../LightningModules/GNNClustering/utils.py:132: RuntimeWarning: invalid value encountered in sqrt
  y = sign*(np.sqrt(r**2 - (x - r*np.cos(theta))**2) - r*np.sin(theta))
/global/u2/r/ryanliu/Tracking-ML-Exa.TrkX/Pipelines/Common_Tracking_Example/notebooks/ITk/../../LightningModules/GNNClustering/utils.py:132: RuntimeWarning: invalid value encountered in sqrt
  y = sign*(np.sqrt(r**2 - (x - r*np.cos(theta))**2) - r*np.sin(theta))
/global/u2/r/ryanliu/Tracking-ML-Exa.TrkX/Pipelines/Common_Tracking_Example/notebooks/ITk/../../LightningModules/GNNClustering/utils.py:132: RuntimeWarning: invalid value encountered in sqrt
  y = sign*(np.sqrt(r**2 - (x - r*np.cos(theta))**2) - r*np.sin(theta))
/global/u2/r/ryanliu/Tracking-ML-Exa.TrkX/Pipelines/Common_Tracking_Example/notebooks/ITk/../../LightningModules/GNNClustering/utils.py:132: RuntimeWarning: invalid value encountered in sqrt
  y = sign*(np.sqrt(


Epoch 26:   0%|          | 0/1020 [00:00<?, ?it/s, loss=0.985, v_num=l3nj]           

/global/u2/r/ryanliu/Tracking-ML-Exa.TrkX/Pipelines/Common_Tracking_Example/notebooks/ITk/../../LightningModules/GNNClustering/utils.py:132: RuntimeWarning: invalid value encountered in sqrt
  y = sign*(np.sqrt(r**2 - (x - r*np.cos(theta))**2) - r*np.sin(theta))
/global/u2/r/ryanliu/Tracking-ML-Exa.TrkX/Pipelines/Common_Tracking_Example/notebooks/ITk/../../LightningModules/GNNClustering/utils.py:132: RuntimeWarning: invalid value encountered in sqrt
  y = sign*(np.sqrt(r**2 - (x - r*np.cos(theta))**2) - r*np.sin(theta))
/global/u2/r/ryanliu/Tracking-ML-Exa.TrkX/Pipelines/Common_Tracking_Example/notebooks/ITk/../../LightningModules/GNNClustering/utils.py:132: RuntimeWarning: invalid value encountered in sqrt
  y = sign*(np.sqrt(r**2 - (x - r*np.cos(theta))**2) - r*np.sin(theta))
/global/u2/r/ryanliu/Tracking-ML-Exa.TrkX/Pipelines/Common_Tracking_Example/notebooks/ITk/../../LightningModules/GNNClustering/utils.py:132: RuntimeWarning: invalid value encountered in sqrt
  y = sign*(np.sqrt(

Epoch 26:   2%|▏         | 18/1020 [00:03<02:55,  5.70it/s, loss=0.977, v_num=l3nj]

## Initialize from trained model

In [ ]:
import wandb
wandb.finish()

with open("dual_embedding_default.yaml") as f:
    hparams = yaml.load(f, Loader=yaml.FullLoader)
    
hparams["use_dual_encoder"] = True

model = VanillaDualEmbedding(hparams)

checkpoint = torch.load("/global/cfs/cdirs/m3443/usr/ryanliu/ITk_embedding/ITk_dual_embedding/3ijb4qnw/checkpoints/last.ckpt")
state_dict = checkpoint["state_dict"]
names = [i for i in state_dict]
for i in names:
    state = state_dict[i]
    i = i.replace("input_layer1", "input_layer2")
    i = i.replace("layers1", "layers2")
    i = i.replace("output_layer1", "output_layer2")
    state_dict[i] = state

model.load_state_dict(state_dict)
del state_dict

In [ ]:
from pytorch_lightning.callbacks import ModelCheckpoint

checkpoint_callback = ModelCheckpoint(
    monitor='pur',
    mode="max",
    save_top_k=2,
    save_last=True)

In [ ]:
logger = WandbLogger(project="ITk_dual_embedding")
trainer = Trainer(gpus=1, max_epochs=hparams["max_epochs"], logger=logger, num_sanity_val_steps=2, callbacks=[checkpoint_callback], log_every_n_steps = 50, default_root_dir="/global/cfs/cdirs/m3443/usr/ryanliu/ITk_embedding/")
trainer.fit(model)